### Imports

In [1]:
import sys
sys.path.append("..")
from definitions import *

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/home/jupyter/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


### Auth to Hugging Face

In [2]:
from huggingface_hub import login
login(token="")

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /tmp/xdg_cache/huggingface/token
Login successful


### Ensuring CUDA

In [3]:
! nvidia-smi

Mon Apr 29 14:02:26 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.08             Driver Version: 535.161.08   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-PCIE-32GB           On  | 00000000:8C:00.0 Off |                  Off |
| N/A   29C    P0              26W / 250W |      4MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
import torch

print(torch.cuda.is_available())
# True

print(torch.cuda.device_count())
# 1

print(torch.cuda.current_device())
# 0

print(torch.cuda.device(0))
# <torch.cuda.device at 0x7efce0b03be0>

torch.cuda.get_device_name(0)
# 'GeForce GTX 950M'

True
1
0


'Tesla V100-PCIE-32GB'

### Try to run MERA lm-eval

In [ ]:
%%time
! CUDA_VISIBLE_DEVICES=0 python3 /home/jupyter/work/resources/MERA/lm-evaluation-harness/main.py --model hf-causal-experimental --model_args pretrained=mistralai/Mistral-7B-v0.1,dtype=auto,max_length=11500 \
--device cuda --output_base_path="$PWD/mera_results/Mistral-7B-v0.1_defaults" --batch_size=1 \
--inference --write_out --no_cache --tasks rummlu --num_fewshot=5 \
--output_path="$PWD/mera_results/Mistral-7B-v0.1_defaults/rummlu_result.json"

Selected Tasks: ['rummlu']
Using `model.prepare_inputs_for_generation` method for `model.forward`.
Searching for good max length. OOM at length 11500
length 5800 not failed but it can be lengthier
length 8650 not failed but it can be lengthier
Searching for good max length. OOM at length 10075
length 9362 not failed but it can be lengthier
Searching for good max length. OOM at length 9718
length 9539 not failed but it can be lengthier
Searching for good max length. OOM at length 9628
Searching for good max length. OOM at length 9583
9560 is good enough
Task: rummlu; number of docs: 961
Running loglikelihood requests
{
  "results": {
    "rummlu": {
      "metric": 0.0,
      "metric_stderr": 0.0
    }
  },
  "versions": {
    "rummlu": 0
  },
  "tasks": {
    "rummlu": 961
  },
  "config": {
    "model": "hf-causal-experimental",
    "model_args": "pretrained=mistralai/Mistral-7B-v0.1,dtype=auto,max_length=11500",
    "num_fewshot": 5,
    "batch_size": "1",
    "batch_sizes": [],
    

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2024-04-29 14:03:34.148799: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
100%|██████████| 3844/3844 [1:50:52<00:00,  1.73s/it]  


### View MERA results

In [8]:
! pwd

/home/jupyter/work/resources/long_context_LLMs/notebooks/metrics_calculation


In [9]:
! ls

__pycache__
knowledge_tests_evaluator.ipynb
llm_helpers.py
mera_evaluation.ipynb
mera_results
mmlu_ru_evaluator.ipynb
open_book_qa_evaluator.ipynb


In [11]:
with open("./mera_results/Mistral-7B-v0.1_defaults/rummlu_result.json", "r") as f:
    rummlu_result = json.load(f)

In [14]:
print(rummlu_result['config']['pretty_env_info'])

PyTorch version: 2.3.0+cu121
Is debug build: False
CUDA used to build PyTorch: 12.1
ROCM used to build PyTorch: N/A

OS: Ubuntu 22.04.2 LTS (x86_64)
GCC version: (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Clang version: Could not collect
CMake version: version 3.25.2
Libc version: glibc-2.35

Python version: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0] (64-bit runtime)
Python platform: Linux-5.13.0-40-generic-x86_64-with-glibc2.35
Is CUDA available: True
CUDA runtime version: 11.8.89
CUDA_MODULE_LOADING set to: LAZY
GPU models and configuration: GPU 0: Tesla V100-PCIE-32GB
Nvidia driver version: 535.161.08
cuDNN version: Probably one of the following:
/usr/lib/x86_64-linux-gnu/libcudnn.so.8.9.0
/usr/lib/x86_64-linux-gnu/libcudnn_adv_infer.so.8.9.0
/usr/lib/x86_64-linux-gnu/libcudnn_adv_train.so.8.9.0
/usr/lib/x86_64-linux-gnu/libcudnn_cnn_infer.so.8.9.0
/usr/lib/x86_64-linux-gnu/libcudnn_cnn_train.so.8.9.0
/usr/lib/x86_64-linux-gnu/libcudnn_ops_infer.so.8.9.0
/usr/lib/x86_64-linux-gnu

In [15]:
with open("./mera_results/Mistral-7B-v0.1_defaults/evaluation_config.json", "r") as f:
    evaluation_config = json.load(f)
print(evaluation_config['pretty_env_info'])

PyTorch version: 2.3.0+cu121
Is debug build: False
CUDA used to build PyTorch: 12.1
ROCM used to build PyTorch: N/A

OS: Ubuntu 22.04.2 LTS (x86_64)
GCC version: (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Clang version: Could not collect
CMake version: version 3.25.2
Libc version: glibc-2.35

Python version: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0] (64-bit runtime)
Python platform: Linux-5.13.0-40-generic-x86_64-with-glibc2.35
Is CUDA available: True
CUDA runtime version: 11.8.89
CUDA_MODULE_LOADING set to: LAZY
GPU models and configuration: GPU 0: Tesla V100-PCIE-32GB
Nvidia driver version: 535.161.08
cuDNN version: Probably one of the following:
/usr/lib/x86_64-linux-gnu/libcudnn.so.8.9.0
/usr/lib/x86_64-linux-gnu/libcudnn_adv_infer.so.8.9.0
/usr/lib/x86_64-linux-gnu/libcudnn_adv_train.so.8.9.0
/usr/lib/x86_64-linux-gnu/libcudnn_cnn_infer.so.8.9.0
/usr/lib/x86_64-linux-gnu/libcudnn_cnn_train.so.8.9.0
/usr/lib/x86_64-linux-gnu/libcudnn_ops_infer.so.8.9.0
/usr/lib/x86_64-linux-gnu

### Download ru_mmlu from Hugging Face

In [28]:
! curl -X GET \
     "https://datasets-server.huggingface.co/rows?dataset=ai-forever%2FMERA&config=bps&split=test&length=1000" >ru_mmlu_dataset_test.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    59  100    59    0     0    103      0 --:--:-- --:--:-- --:--:--   103


In [29]:
with open("./ru_mmlu_dataset_test.json", "r") as f:
    ru_mmlu_dataset_test = json.load(f)

In [30]:
ru_mmlu_dataset_test.keys()

dict_keys(['error'])

In [32]:
ru_mmlu_dataset_test['error']

"Parameter 'length' must not be greater than 100"

In [31]:
ru_mmlu_dataset_test['num_rows_per_page']

KeyError: 'num_rows_per_page'

In [25]:
ru_mmlu_dataset_test['rows'].__len__()

100

In [26]:
ru_mmlu_dataset_test['features']

[{'feature_idx': 0,
  'name': 'instruction',
  'type': {'dtype': 'string', '_type': 'Value'}},
 {'feature_idx': 1,
  'name': 'inputs',
  'type': {'dtype': 'string', '_type': 'Value'}},
 {'feature_idx': 2,
  'name': 'outputs',
  'type': {'dtype': 'string', '_type': 'Value'}},
 {'feature_idx': 3,
  'name': 'meta',
  'type': {'id': {'dtype': 'int32', '_type': 'Value'}}}]

In [37]:
from datasets import load_dataset

dataset = load_dataset("ai-forever/MERA", "rummlu", split="test")

In [42]:
for x in dataset:
    print(x['outputs'])
    break

### Calculate how many correct answers

In [5]:
# with open("./mera_results/Mistral-7B-v0.1_defaults/lm_harness_logs_rummlu/input_docs.json", "r") as f:
    # input_docs = json.load(f)
with open("./mera_results/Mistral-7B-v0.1_defaults/lm_harness_logs_rummlu/output_answers.json", "r") as f:
    output_answers = json.load(f)

In [8]:
for output_answer_key, output_answer_value, in output_answers.items():
    print(output_answer_key, output_answer_value[0])
    # print(output_answers.keys())
    break

16 [0, -837.1574096679688]


### Try to convert results to submission

In [36]:
! python3 /home/jupyter/work/resources/MERA/lm-evaluation-harness/scripts/log_to_submission.py --outputs_dir .

Traceback (most recent call last):
  File "/home/jupyter/work/resources/MERA/lm-evaluation-harness/scripts/log_to_submission.py", line 383, in <module>
    main()
  File "/home/jupyter/work/resources/MERA/lm-evaluation-harness/scripts/log_to_submission.py", line 376, in main
    _ = create_submission(args.outputs_dir, args.dst_dir)
  File "/home/jupyter/work/resources/MERA/lm-evaluation-harness/scripts/log_to_submission.py", line 351, in create_submission
    _ = task.convert()
  File "/home/jupyter/work/resources/MERA/lm-evaluation-harness/scripts/log_to_submission.py", line 82, in convert
    submission = self.outputs_to_submission(load_json(self.outputs_path))
  File "/home/jupyter/work/resources/MERA/lm-evaluation-harness/scripts/log_to_submission.py", line 28, in load_json
    with open(path, "r", encoding="utf-8") as file:
FileNotFoundError: [Errno 2] No such file or directory: './lm_harness_logs_bps/output_answers.json'


Exception: Process exited with code 1